In [14]:
import os, pprint
pprint.pprint({k:v for k,v in os.environ.items() if 'PROXY' in k.upper() or 'NO_PROXY' in k.upper()})

{'PROXY_API_PORT': 'tcp://10.233.60.164:8001',
 'PROXY_API_PORT_8001_TCP': 'tcp://10.233.60.164:8001',
 'PROXY_API_PORT_8001_TCP_ADDR': '10.233.60.164',
 'PROXY_API_PORT_8001_TCP_PORT': '8001',
 'PROXY_API_PORT_8001_TCP_PROTO': 'tcp',
 'PROXY_API_SERVICE_HOST': '10.233.60.164',
 'PROXY_API_SERVICE_PORT': '8001',
 'PROXY_PUBLIC_PORT': 'tcp://10.233.12.107:80',
 'PROXY_PUBLIC_PORT_80_TCP': 'tcp://10.233.12.107:80',
 'PROXY_PUBLIC_PORT_80_TCP_ADDR': '10.233.12.107',
 'PROXY_PUBLIC_PORT_80_TCP_PORT': '80',
 'PROXY_PUBLIC_PORT_80_TCP_PROTO': 'tcp',
 'PROXY_PUBLIC_SERVICE_HOST': '10.233.12.107',
 'PROXY_PUBLIC_SERVICE_PORT': '80',
 'PROXY_PUBLIC_SERVICE_PORT_HTTP': '80'}


In [1]:
!curl -sS -m 5 -w 'HTTP %{http_code}\n' \
  http://alphafold.protein-binder-design:8081/v1/health/ready

curl: (28) Connection timed out after 5001 milliseconds
HTTP 000


In [2]:
!curl -sS -m 5 -w 'HTTP %{http_code}\n' \
  http://alphafold.protein-binder-design:8081/v1/health/ready

{"status":"ready"}HTTP 200


In [3]:
import os, requests

# Map service -> (namespace, service-port)
SERVICES = {
    "alphafold": ("protein-binder-design", 8081),
    "rfdiffusion": ("protein-binder-design", 8082),
    "proteinmpnn": ("protein-binder-design", 8083),
    "alphafold-multimer": ("protein-binder-design", 8084),
}

# One session, and ignore any env HTTP proxies inside the pod
SESSION = requests.Session()
SESSION.trust_env = False   # important in clusters with proxies

def call(service, path, *, timeout=10, auth=False, json=None):
    ns, port = SERVICES[service]
    base = f"http://{service}.{ns}:{port}"
    headers = {}
    if auth and (k := os.getenv("NGC_API_KEY")):
        headers["Authorization"] = f"Bearer {k}"
    r = SESSION.get(base + path, headers=headers, timeout=timeout) if json is None \
        else SESSION.post(base + path, headers=headers, json=json, timeout=timeout)
    print(f"{service} {path} -> {r.status_code}")
    return r

# quick health checks
for s in SERVICES:
    call(s, "/v1/health/ready")
    call(s, "/v1/health/live")

alphafold /v1/health/ready -> 200
alphafold /v1/health/live -> 200
rfdiffusion /v1/health/ready -> 200
rfdiffusion /v1/health/live -> 200
proteinmpnn /v1/health/ready -> 200
proteinmpnn /v1/health/live -> 200
alphafold-multimer /v1/health/ready -> 200
alphafold-multimer /v1/health/live -> 200
